# Regression Exercise

In [20]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

C:\Users\Jayuk\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
housing = pd.read_csv('cal_housing_clean.csv')

In [5]:
housing.head(5)

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [6]:
X = housing.drop(['medianHouseValue'], axis=1)

In [7]:
Y = housing['medianHouseValue']

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3,random_state=101)

In [15]:
scaler = MinMaxScaler()

In [16]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [17]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [18]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

In [19]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [24]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [25]:
feat_cols = [ age,rooms,bedrooms,pop,households,income]

In [27]:
input_func = tf.estimator.inputs.pandas_input_fn(
    x=X_train,
    y=Y_train ,
    batch_size=10,
    num_epochs=1000,
    shuffle=True)

In [28]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Jayuk\\AppData\\Local\\Temp\\tmp09l4d42q', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E03865AA58>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [29]:
model.train(input_fn=input_func, steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Jayuk\AppData\Local\Temp\tmp09l4d42q\model.ckpt.
INFO:tensorflow:loss = 707155460000.0, step = 0
INFO:tensorflow:global_step/sec: 169.807
INFO:tensorflow:loss = 286807260000.0, step = 100 (0.590 sec)
INFO:tensorflow:global_step/sec: 192.295
INFO:tensorflow:loss = 832843900000.0, step = 200 (0.521 sec)
INFO:tensorflow:global_step/sec: 197.217
INFO:tensorflow:loss = 490933060000.0, step = 300 (0.507 sec)
INFO:tensorflow:global_step/sec: 199.177
INFO:tensorflow:loss = 392430350000.0, step = 400 (0.501 sec)
INFO:tensorflow:global_step/sec: 203.011
INFO:tensorflow:loss = 629000600000.0, step = 500 (0.494 sec)
INFO:tensorflow:global_step/sec: 199.773
INFO:tensorflow:loss = 512653660000.0, step = 600 (0.49

In [30]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
    x=X_test,
    batch_size=10,
    num_epochs=1,
    shuffle=False)

In [31]:
predictions = model.predict(predict_input_func)

In [32]:
predictions = list(predictions)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Jayuk\AppData\Local\Temp\tmp09l4d42q\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [33]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [34]:
from sklearn.metrics import mean_squared_error

In [35]:
mean_squared_error(y_test,final_preds)**0.5

105772.51138140228